## Data provided by Marvel. © 2019 Marvel

Searching Api: My audience is exigent, they are teenagers. I must serve the best.

# Project Web

## by Jordi Marsal<br><br>

# A  -  API<br><br>

## Marvel API<br>
### developper account:
#### https://developer.marvel.com/account
__Rate Limits:__<br>
3000 calls/day<br><br>
__Attribution:__<br>
Data provided by Marvel. © 2019 Marvel<br><br>
__Linking to Marvel:__<br>
Linking from your application and web pages to Marvel helps us understand how applications are being used by end-users, which applications are most successful, and which presentations of Marvel content is most impactful, all of which helps us improve our API products.<br><br>
If a URL array is not present for an entity simply link to the Marvel.com home page (http://marvel.com) in your attribution notice.<br><br>

GENERAL API INFORMATION<br><br>
The Marvel Comics __API__ is a __RESTful__ service which provides methods for accessing specific resources at canonical URLs and for searching and filtering sets of resources by various criteria. All representations are encoded as __JSON__ objects.

### Imports

In [1]:
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
from functools import reduce
from datetime import datetime
#import hashlib
from bs4 import BeautifulSoup 
import re
import random
#import urllib.parse
import ipywidgets as widgets
from IPython.display import display, clear_output, Image, IFrame, HTML
import jupyter_dashboards

In [2]:
struct = {
    'endpoint':'http://gateway.marvel.com:80',
    'characters': '/v1/public/characters',
    'comics':'/v1/public/comics',
    'creators':'/v1/public/creators',
    'events':'/v1/public/events',
    'series':'/v1/public/series',
    'stories':'/v1/public/stories',
    'pcharacters':['name','nameStartsWith','limit','modifiedSince','comics','events','series','stories']
}
call = 'https://gateway.marvel.com:443/v1/public/characters?nameStartsWith=Spider&limit=25&apikey=e27f579a21906a26a77b3e6fce0cc65d'
api_key = 'e27f579a21906a26a77b3e6fce0cc65d'
private_key = '44e123c5770ed2bee1cf1c524c61e5dc8ab3d739'



In [3]:
class Aux:
    
    def timestamp():
        now = datetime.now()
        return ''.join(str(datetime.timestamp(now)).split('.'))

    def string(string, var):
        return string % var
    
    def get_state(res):
        #print("Status code = "+str(res.status_code))
        #print("Message = "+str(res.text))
        if res.status_code >=200 and res.status_code <300:
            print("  This is OK")
        if res.status_code >=300 and res.status_code <400:
            print("  This is a redirection")
        if res.status_code >=400 and res.status_code <500:
            print("  This is a Client problem")
        if res.status_code >=500 and res.status_code <600:
            print("  This is a Server problem")
        #print(res.headers)
        #print(res.content)
        #print(res.json())
            
    #hash - a md5 digest of the ts parameter, your private key and your public key (e.g. md5(ts+privateKey+publicKey)
    def hash_md5(string):
        return hashlib.md5(string.encode('utf-8')).hexdigest()
    
    def image(url):
        return Image(url, width=100, height=100)

class API:
    def __init__(self, struct, api_key, priv_key):
        self.struct = struct
        self.endpoint = struct['endpoint']
        self.end = '/'
        self.api_key = api_key
        self.priv_key = priv_key
        self.err = []
        self.headers = {'Accept': '*/*'}
    
    def forced(self, params):
        if 'limit' not in params:
            params['limit']='10'
        t = Aux.timestamp()
        params['ts'] = t
        params['apikey'] = self.api_key
        params['hash'] = Aux.hash_md5(t+self.priv_key+self.api_key)
        print(params)
        return params
    
    def _validate(self, path, params):
        if path not in self.struct:
            self.err.append(Aux.string("Path %s not exists",path))
        if not self.err:
            pset = "%s%s" % ('p', path)
            pparams = self.struct[pset]
            for p in params:
                if p not in pparams:
                    self.err.append(Aux.string("Parameter %s not exists",p))
        if self.err:
            print(self.err)
            
        return len(self.err)==0
    
    def _get_url(self, path, params):
        url = self.endpoint + self.struct[path]
        params = self.forced(params)
        if params:
            url += "?"
            for i,(key, value) in enumerate(params.items()):
                end = '$' if i < len(params) -1 else ''
                url += "%s=%s%s" % (key, value, end)
        
        print('URL:',url)
        return url
            
    def _connect(self, url):
        headers = self.get_random_ua()
        return requests.get(url, headers = headers)
        
    def get_json(self, url):
        response = self._connect(url)
        Aux.get_state(response)
        return self._connect(url).json()
    
    def get(self, path, params={}):
        if self._validate(path, params):
            self.get_json(self._get_url(path,params))
            
    def clear_err(self):
        self.err = []
        
    def get_random_ua(self):
        heads =[
            'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Mobile Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:67.0) Gecko/20100101 Firefox/67.0',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15'
        ]
        headers = {'User-Agent': '','Accept': '*/*'}
        sample = random.sample(heads,1)
        headers['User-Agent'] = sample[0]
        return headers

In [4]:
api = API(struct, api_key, private_key)

In [5]:
#json = api.get('characters')
#df = pd.DataFrame(json)
#df.head()


In [6]:

print(Aux.timestamp())

1563730502245491


In [7]:
omdb = 'http://www.omdbapi.com/?apikey=f2f47e82&t=Spider&y=2019'

ojson = api.get_json(omdb)

ordb = pd.DataFrame(ojson)
print(ojson)
ordb

  This is OK
{'Title': 'Spider-Man: Far from Home', 'Year': '2019', 'Rated': 'PG-13', 'Released': '02 Jul 2019', 'Runtime': '129 min', 'Genre': 'Action, Adventure, Sci-Fi', 'Director': 'Jon Watts', 'Writer': 'Chris McKenna, Stan Lee (based on the Marvel comic book by), Steve Ditko (based on the Marvel comic book by), Erik Sommers (screenplay by)', 'Actors': 'Tom Holland, Samuel L. Jackson, Jake Gyllenhaal, Marisa Tomei', 'Plot': 'Following the events of Avengers: Endgame (2019), Spider-Man must step up to take on new threats in a world that has changed forever.', 'Language': 'English, Italian, Czech', 'Country': 'USA', 'Awards': 'N/A', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMGZlNTY1ZWUtYTMzNC00ZjUyLWE0MjQtMTMxN2E3ODYxMWVmXkEyXkFqcGdeQXVyMDM2NDM2MQ@@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.0/10'}, {'Source': 'Metacritic', 'Value': '69/100'}], 'Metascore': '69', 'imdbRating': '8.0', 'imdbVotes': '110,832', 'imdbID': 'tt6320628', 'Type': '

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Spider-Man: Far from Home,2019,PG-13,02 Jul 2019,129 min,"Action, Adventure, Sci-Fi",Jon Watts,"Chris McKenna, Stan Lee (based on the Marvel c...","Tom Holland, Samuel L. Jackson, Jake Gyllenhaa...",Following the events of Avengers: Endgame (201...,...,69,8.0,"110,832",tt6320628,movie,N/A,N/A,Sony Pictures,N/A,True
1,Spider-Man: Far from Home,2019,PG-13,02 Jul 2019,129 min,"Action, Adventure, Sci-Fi",Jon Watts,"Chris McKenna, Stan Lee (based on the Marvel c...","Tom Holland, Samuel L. Jackson, Jake Gyllenhaa...",Following the events of Avengers: Endgame (201...,...,69,8.0,"110,832",tt6320628,movie,N/A,N/A,Sony Pictures,N/A,True


In [8]:
url = 'https://es.wikipedia.org/wiki/Universo_cinematogr%C3%A1fico_de_Marvel'
html = requests.get(url).content

In [9]:
soup = BeautifulSoup(html,'lxml')

table = soup.find_all('table',{'class':'wikitable plainrowheaders'})[0]

In [10]:
#print(table)

def no_brackets(string):
    r = re.sub('\[\d+\]', '',string)
    return r

rows = table.find_all('tr')

names = [no_brackets(span.a.text) for span in rows if span.a and no_brackets(span.a.text)]

print (names)

['Iron Man', 'The Incredible Hulk', 'Iron Man 2', 'Thor', 'Capitán América: el primer vengador', 'The Avengers', 'Iron Man 3', 'Thor: The Dark World', 'Captain America: The Winter Soldier', 'Guardianes de la Galaxia', 'Avengers: Age of Ultron', 'Ant-Man', 'Capitán América: Civil War', 'Doctor Strange', 'Guardianes de la Galaxia Vol. 2', 'Spider-Man: Homecoming', 'Thor: Ragnarok', 'Black Panther', 'Avengers: Infinity War', 'Ant-Man and the Wasp', 'Capitana Marvel', 'Avengers: Endgame', 'Spider-Man: Lejos de casa', 'Black Widow', 'Eternals']


In [11]:
#omdb = 'http://www.omdbapi.com/?apikey=f2f47e82&t=Spider&y=2019'
omdb = 'http://www.omdbapi.com/?apikey=f2f47e82&t=Iron Man'
#ojson = api.get_json(omdb)
#print(ojson)

def get_film(title):
    base_url = 'http://www.omdbapi.com?apikey=f2f47e82'
    url = base_url+"&t="+title #urllib.parse.quote(title)
    #url = urllib.parse.quote(url)
    print(url)
    fjson = api.get_json(url)

dfb = pd.DataFrame()
for t in names[:2]:
    print(t)
    fjson = get_film(t)
    print(fjson)
    #dfb = pd.concat(fjson,axis=0)
#dfb

Iron Man
http://www.omdbapi.com?apikey=f2f47e82&t=Iron Man
  This is OK
None
The Incredible Hulk
http://www.omdbapi.com?apikey=f2f47e82&t=The Incredible Hulk
  This is OK
None


In [12]:
giphy_key = 'qalYklnRNsDnfcN8yepwUSH8NfIX4SqL'
endpoint = 'api.giphy.com/v1/gifs/search'

def get_gif(search):
    base_url = 'http://api.giphy.com/v1/gifs/search?'
    url = base_url+"&q="+search+'&api_key='+ giphy_key + '&limit=5' #urllib.parse.quote(title)
    print(url)
    fjson = api.get_json(url)
    return fjson

#gjson=get_gif('Spider-Man')
#gif = pd.DataFrame(gjson['data'])
#gif.head()


for i,t in enumerate(names[:3]):
    print(t)
    fjson = get_gif(t)
    if i==0:
        df1 = pd.DataFrame(json_normalize(fjson['data']))
    else:
        df = pd.DataFrame(json_normalize(fjson['data']))
        df1 = pd.concat([df1, df], axis='rows')
    
df1 = df1.reset_index(drop=True)

Iron Man
http://api.giphy.com/v1/gifs/search?&q=Iron Man&api_key=qalYklnRNsDnfcN8yepwUSH8NfIX4SqL&limit=5
  This is OK
The Incredible Hulk
http://api.giphy.com/v1/gifs/search?&q=The Incredible Hulk&api_key=qalYklnRNsDnfcN8yepwUSH8NfIX4SqL&limit=5
  This is OK
Iron Man 2
http://api.giphy.com/v1/gifs/search?&q=Iron Man 2&api_key=qalYklnRNsDnfcN8yepwUSH8NfIX4SqL&limit=5


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



  This is OK


In [13]:
df1.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
analytics.onclick.url,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...
analytics.onload.url,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...
analytics.onsent.url,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...,https://giphy-analytics.giphy.com/simple_analy...
bitly_gif_url,https://gph.is/1cdvrJy,https://gph.is/1TM0S3J,https://gph.is/2w1pCEt,https://gph.is/1BLGQSu,https://gph.is/YBYwt3,https://gph.is/11jMYiz,https://gph.is/1F78wn2,https://gph.is/YBD4EB,https://gph.is/1sCx57M,https://gph.is/1B8xNaJ,https://gph.is/XKfxVH,https://gph.is/Z130JA,https://gph.is/1ilC6ap,https://gph.is/1fgoYWb,https://gph.is/1cGSpgC
bitly_url,https://gph.is/1cdvrJy,https://gph.is/1TM0S3J,https://gph.is/2w1pCEt,https://gph.is/1BLGQSu,https://gph.is/YBYwt3,https://gph.is/11jMYiz,https://gph.is/1F78wn2,https://gph.is/YBD4EB,https://gph.is/1sCx57M,https://gph.is/1B8xNaJ,https://gph.is/XKfxVH,https://gph.is/Z130JA,https://gph.is/1ilC6ap,https://gph.is/1fgoYWb,https://gph.is/1cGSpgC
content_url,,,,,,,,,,,,,,,
embed_url,https://giphy.com/embed/UJ5I7921pAOEU,https://giphy.com/embed/l3975CZuyQgoNVuOA,https://giphy.com/embed/iHTRo3hYZGhZS,https://giphy.com/embed/OPOIcmwa6Ew2A,https://giphy.com/embed/bCvO8biVh8WyI,https://giphy.com/embed/XKL1MHvk47OXC,https://giphy.com/embed/Gwtv1sXXnunMA,https://giphy.com/embed/1zv2iEutcQAqA,https://giphy.com/embed/3aUGq69kSLj4k,https://giphy.com/embed/aFVshRs8GMqgo,https://giphy.com/embed/LNKBLt58WwBJC,https://giphy.com/embed/PnwcgZJZ8jdbq,https://giphy.com/embed/lXo8uSnIkaB9e,https://giphy.com/embed/qXd2HVXvg9Fu,https://giphy.com/embed/8ltyoIL6d7uZG
id,UJ5I7921pAOEU,l3975CZuyQgoNVuOA,iHTRo3hYZGhZS,OPOIcmwa6Ew2A,bCvO8biVh8WyI,XKL1MHvk47OXC,Gwtv1sXXnunMA,1zv2iEutcQAqA,3aUGq69kSLj4k,aFVshRs8GMqgo,LNKBLt58WwBJC,PnwcgZJZ8jdbq,lXo8uSnIkaB9e,qXd2HVXvg9Fu,8ltyoIL6d7uZG
images.480w_still.height,240,480,270,270,411,370,406,225,360,290,404,490,305,392,480
images.480w_still.url,https://med

![images.preview_gif.url](https://media2.giphy.com/media/UJ5I7921pAOEU/giphy-preview.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy-preview.gif
)

In [14]:
#mini = images.preview_gif.url
#medium= images.downsized_medium.url
# images.downsized_medium.url
url = 'https://giphy.com/embed/UJ5I7921pAOEU'
url2 =  df1['embed_url'][0]
print(url2)
#IFrame(src=url, width=350, height=200)
IFrame(src=url2, width=350, height=200)

https://giphy.com/embed/UJ5I7921pAOEU


![Jordi](https://media2.giphy.com/media/UJ5I7921pAOEU/giphy.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy.gif
)

In [15]:
df1.shape[0]

15

In [16]:
def uniques_values_all(serie):
    ALL='ALL'
    uni =  serie.unique().tolist()
    return uni
titles = uniques_values_all(df1.title)
urls = uniques_values_all(df1['images.downsized_medium.url'])
titles

['https://media2.giphy.com/media/UJ5I7921pAOEU/giphy.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy.gif',
 'https://media2.giphy.com/media/l3975CZuyQgoNVuOA/giphy.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy.gif',
 'https://media0.giphy.com/media/iHTRo3hYZGhZS/giphy-downsized-medium.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy-downsized-medium.gif',
 'https://media3.giphy.com/media/OPOIcmwa6Ew2A/giphy.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy.gif',
 'https://media1.giphy.com/media/bCvO8biVh8WyI/giphy.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy.gif',
 'https://media0.giphy.com/media/XKL1MHvk47OXC/giphy.gif?cid=a937cb845d3478d75376336c679a3b28&rid=giphy.gif',
 'https://media3.giphy.com/media/Gwtv1sXXnunMA/giphy.gif?cid=a937cb845d3478d75376336c679a3b28&rid=giphy.gif',
 'https://media3.giphy.com/media/1zv2iEutcQAqA/giphy.gif?cid=a937cb845d3478d75376336c679a3b28&rid=giphy.gif',
 'https://media0.giphy.com/media/3aUGq69kSLj4k/giphy.gif?cid=a937cb845d3478d753763

In [17]:


dropd_gif = widgets.Dropdown(options = titles)
output = widgets.Output()

In [30]:
def df_filtering(gif,usertype):
    output.clear_output()
    df_filter = df1[(df1.title == gif)]
    url = urls[titles.index(gif)]
    url2 =  df1['images.downsized_medium.url'][0]
    #print(gif,url)
    with output:
        display(HTML("""<div style=\"display:inline-flex\">
        <img height=\"100\" src='%s'>
        &nbsp;
        <img style=\"margin: 0;\" height=\"100\" src='%s'></div>""" %(url,url2)))

![images.preview_gif.url](url
)

In [19]:
def dr_gif_handler(change):
    df_filtering(change.new,dropd_gif.value)
    
dropd_gif.observe(dr_gif_handler,names='value')
    
display(dropd_gif)

Dropdown(options=('iron man GIF', 'suit up iron man GIF by 100% Soft', 'iron man bathroom GIF', 'iron man marv…

In [20]:
display(output)

Output()

In [21]:


from string import Template

img_list = urls

fig_template = Template('''<figure>
    <img src="$fn" alt='$capt'>
    <figcaption> Figure $fig_idx. $capt</figcaption>
</figure>'''
)

for idx, img in enumerate(img_list, 1):
    print (fig_template.substitute(fn=img, capt=img, fig_idx=idx))

<figure>
    <img src="https://media2.giphy.com/media/UJ5I7921pAOEU/giphy.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy.gif" alt='https://media2.giphy.com/media/UJ5I7921pAOEU/giphy.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy.gif'>
    <figcaption> Figure 1. https://media2.giphy.com/media/UJ5I7921pAOEU/giphy.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy.gif</figcaption>
</figure>
<figure>
    <img src="https://media2.giphy.com/media/l3975CZuyQgoNVuOA/giphy.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy.gif" alt='https://media2.giphy.com/media/l3975CZuyQgoNVuOA/giphy.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy.gif'>
    <figcaption> Figure 2. https://media2.giphy.com/media/l3975CZuyQgoNVuOA/giphy.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy.gif</figcaption>
</figure>
<figure>
    <img src="https://media0.giphy.com/media/iHTRo3hYZGhZS/giphy-downsized-medium.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy-downsized-medium.gif" alt='https://media0.giphy.